In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import nltk
import random
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import cross_val_score
from sklearn import model_selection
from nltk.stem import PorterStemmer, LancasterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer,TfidfTransformer
from xgboost import XGBClassifier
from xgboost import plot_importance
import xgboost as xgb
from matplotlib import pyplot
from sklearn.calibration import CalibratedClassifierCV
import lightgbm as lgb
import datetime 
import random
import pprint as pp
import pickle

import numpy as np
np.random.seed( 42 )
random.seed(42)

In [2]:
train = pd.read_pickle('data/trainStemmed')
test = pd.read_pickle('data/testStemmed')

textColumnNames = ['PText', 'PTextStem']

In [3]:
countVect = CountVectorizer()
wordCountPerDoc = countVect.fit_transform(train['PTextStem'])
wordCountPerDoc

tTransformer = TfidfTransformer(smooth_idf=False)
tfidf = tTransformer.fit_transform(wordCountPerDoc)
tfidf = tfidf.toarray()
print(wordCountPerDoc.shape)
print(tfidf.shape)

(3321, 112037)
(3321, 112037)


In [6]:
def wordsByTFFrequency(countVect, tfidf, wordCountPerDoc) : 
    su = wordCountPerDoc.sum(axis=0).tolist()[0]
    sutf = tfidf.sum(axis=0).tolist()
    
    mostFrequentWords = [(word, su[idx], sutf[idx]) for word, idx in countVect.vocabulary_.items()]
    mostFrequentWords = sorted(mostFrequentWords, key=lambda x : -x[1])
    mostFrequentWordsTF = sorted(mostFrequentWords, key=lambda x : -x[2])
    
    return mostFrequentWordsTF

x = wordsByTFFrequency(countVect, tfidf, wordCountPerDoc)

# print(r[:5])
x[:5]

[('mutat', 364032, 404.281295220104),
 ('cell', 297391, 278.8976113833016),
 ('activ', 186274, 196.8025388951524),
 ('brca1', 53994, 161.04688306415594),
 ('variant', 89041, 152.8287228708867)]

In [10]:
vectorizers = {}
transformers = {}
tfidfs = {}
wordCounts = {}
sortedTf = {}
words = {}

for textModification in textColumnNames: # PText, PTextStem
    for c in train['Class'].unique():
        cn = textModification+str(c)
        print(cn)
        vectorizers[cn] = CountVectorizer()
        wordCounts[cn] = vectorizers[cn].fit_transform(train[train['Class']==c][textModification])

        transformers[cn] = TfidfTransformer(smooth_idf=False)
        tfidfs[cn] = transformers[cn].fit_transform(wordCounts[cn])
        tfidfs[cn] = tfidfs[cn].toarray()

        sortedTf[cn] = wordsByTFFrequency(vectorizers[cn], tfidfs[cn],wordCounts[cn])
        words[cn] = [r[0] for r in sortedTf[cn]]


PText1
PText2
PText3
PText4
PText5
PText6
PText7
PText8
PText9
PTextStem1
PTextStem2
PTextStem3
PTextStem4
PTextStem5
PTextStem6
PTextStem7
PTextStem8
PTextStem9


## Intersections

In [12]:
intersect = {}
for textModification in textColumnNames: # PText, PTextStem
    intersect[textModification] = set(words[textModification+'1'])
    for c in range(2,10):
        intersect[textModification] = intersect[textModification] & set(words[textModification+str(c)])
    print(len(intersect[textModification]))
    
with open('data/fullIntersect.pickle', 'wb') as handle:
    pickle.dump(intersect, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('data/fullIntersect.pickle', 'rb') as handle:
#     b = pickle.load(handle)

4364
3103


In [16]:
# Remove words from intersect
wordsWithoutIntesect = {}
for textModification in textColumnNames: # PText, PTextStem
    for c in train['Class'].unique():
        cn = textModification+str(c)
        wordsWithoutIntesect[cn] = [w for w in sortedTf[cn] if w not in intersect]
        print(cn)

PText1
PText2
PText3
PText4
PText5
PText6
PText7
PText8
PText9
PTextStem1
PTextStem2
PTextStem3
PTextStem4
PTextStem5
PTextStem6
PTextStem7
PTextStem8
PTextStem9


In [18]:
def frequentWordsByText(text) :
#     cleanText = getTerms(text)
    countVect = CountVectorizer()
    wordCountPerDoc = countVect.fit_transform([text])

    l = len(text.split())
    
    mostFrequentWords = [(word, wordCountPerDoc[0,idx]/l) for word, idx in countVect.vocabulary_.items()]

    mostFrequentWords = sorted(mostFrequentWords, key=lambda x : -x[1])

    return mostFrequentWords

frequentWordsByText(train['PText'][5])

[('mutations', 0.039174719247845394),
 ('cbl', 0.031600940193261949),
 ('binding', 0.016714546879080701),
 ('cancer', 0.016192217289109429),
 ('stability', 0.010446591799425437),
 ('effects', 0.0086184382345259863),
 ('fig', 0.0083572734395403504),
 ('protein', 0.0078349438495690787),
 ('activity', 0.0078349438495690787),
 ('table', 0.0070514494646121703),
 ('cells', 0.0067902846696265345),
 ('state', 0.0067902846696265345),
 ('states', 0.0065291198746408987),
 ('egfr', 0.0057456254896839903),
 ('proteins', 0.0057456254896839903),
 ('damaging', 0.0054844606946983544),
 ('energy', 0.0054844606946983544),
 ('methods', 0.0052232958997127186),
 ('changes', 0.0049621311047270827),
 ('mutant', 0.0047009663097414469),
 ('cell', 0.0047009663097414469),
 ('random', 0.0047009663097414469),
 ('ubiquitination', 0.0047009663097414469),
 ('mutation', 0.0044398015147558111),
 ('figure', 0.0041786367197701752),
 ('mutants', 0.0041786367197701752),
 ('activation', 0.0041786367197701752),
 ('structures'

In [19]:
for textModification in textColumnNames: # PText, PTextStem
#     train[textModification+'fw'] = train[textModification].apply(lambda x : [wordTuple[0] for wordTuple in frequentWordsByText(x)])
    train[textModification+'wf'] = train[textModification].apply(lambda x : [wordTuple for wordTuple in frequentWordsByText(x)])
    print('done train')
#     test[textModification+'fw'] = test[textModification].apply(lambda x : [wordTuple[0] for wordTuple in frequentWordsByText(x)])
    test[textModification+'wf'] = test[textModification].apply(lambda x : [wordTuple for wordTuple in frequentWordsByText(x)])
    print('done test')

done train
done test
done train
done test


In [20]:
train['PTextwf'][:3]

0    [(cyclin, 0.0241061755146), (cdk10, 0.02085590...
1    [(mutations, 0.0201503759398), (lung, 0.016842...
2    [(mutations, 0.0201503759398), (lung, 0.016842...
Name: PTextwf, dtype: object

In [30]:
def overlapRange(fw, popularWords):
    wordsFromFreq = [w[0] for w in fw]
    n = len(list(set(wordsFromFreq) & set(popularWords)))
    return float(n)

nboverlap = 250
for textModification in textColumnNames: # PText, PTextStem
    for nboverlap in range(20,301,20):
        for i in range(1, 10):
            columnName = textModification + 'Overlap' + str(nboverlap) + 'C' + str(i)
            ignoreWords = []
            popularWords=[w for w in words[textModification + str(i)] if w not in ignoreWords][:nboverlap]

            train[columnName] = train[textModification + 'wf'].apply(lambda x: overlapRange(x[:nboverlap], popularWords))
            test[columnName] = test[textModification + 'wf'].apply(lambda x: overlapRange(x[:nboverlap], popularWords))
            print(columnName)


PTextOverlap20C1
PTextOverlap20C2
PTextOverlap20C3
PTextOverlap20C4
PTextOverlap20C5
PTextOverlap20C6
PTextOverlap20C7
PTextOverlap20C8
PTextOverlap20C9
PTextOverlap40C1
PTextOverlap40C2
PTextOverlap40C3
PTextOverlap40C4
PTextOverlap40C5
PTextOverlap40C6
PTextOverlap40C7
PTextOverlap40C8
PTextOverlap40C9
PTextOverlap60C1
PTextOverlap60C2
PTextOverlap60C3
PTextOverlap60C4
PTextOverlap60C5
PTextOverlap60C6
PTextOverlap60C7
PTextOverlap60C8
PTextOverlap60C9
PTextOverlap80C1
PTextOverlap80C2
PTextOverlap80C3
PTextOverlap80C4
PTextOverlap80C5
PTextOverlap80C6
PTextOverlap80C7
PTextOverlap80C8
PTextOverlap80C9
PTextOverlap100C1
PTextOverlap100C2
PTextOverlap100C3
PTextOverlap100C4
PTextOverlap100C5
PTextOverlap100C6
PTextOverlap100C7
PTextOverlap100C8
PTextOverlap100C9
PTextOverlap120C1
PTextOverlap120C2
PTextOverlap120C3
PTextOverlap120C4
PTextOverlap120C5
PTextOverlap120C6
PTextOverlap120C7
PTextOverlap120C8
PTextOverlap120C9
PTextOverlap140C1
PTextOverlap140C2
PTextOverlap140C3
PTextOverl

In [31]:
for textModification in textColumnNames: # PText, PTextStem
    for nboverlap in range(20,301,20):
        for i in range(1, 10):
            columnName = textModification + 'OverlapIntersect' + str(nboverlap) + 'C' + str(i)
            ignoreWords = intersect[textModification]
            popularWords=[w for w in words[textModification + str(i)] if w not in ignoreWords][:nboverlap]

            train[columnName] = train[textModification + 'wf'].apply(lambda x: overlapRange(x[:nboverlap], popularWords))
            test[columnName] = test[textModification + 'wf'].apply(lambda x: overlapRange(x[:nboverlap], popularWords))
            print(columnName)

PTextOverlapIntersect20C1
PTextOverlapIntersect20C2
PTextOverlapIntersect20C3
PTextOverlapIntersect20C4
PTextOverlapIntersect20C5
PTextOverlapIntersect20C6
PTextOverlapIntersect20C7
PTextOverlapIntersect20C8
PTextOverlapIntersect20C9
PTextOverlapIntersect40C1
PTextOverlapIntersect40C2
PTextOverlapIntersect40C3
PTextOverlapIntersect40C4
PTextOverlapIntersect40C5
PTextOverlapIntersect40C6
PTextOverlapIntersect40C7
PTextOverlapIntersect40C8
PTextOverlapIntersect40C9
PTextOverlapIntersect60C1
PTextOverlapIntersect60C2
PTextOverlapIntersect60C3
PTextOverlapIntersect60C4
PTextOverlapIntersect60C5
PTextOverlapIntersect60C6
PTextOverlapIntersect60C7
PTextOverlapIntersect60C8
PTextOverlapIntersect60C9
PTextOverlapIntersect80C1
PTextOverlapIntersect80C2
PTextOverlapIntersect80C3
PTextOverlapIntersect80C4
PTextOverlapIntersect80C5
PTextOverlapIntersect80C6
PTextOverlapIntersect80C7
PTextOverlapIntersect80C8
PTextOverlapIntersect80C9
PTextOverlapIntersect100C1
PTextOverlapIntersect100C2
PTextOverl

In [32]:
def gboost1(df, df_test, features):
    print('\n##################\nXGBoost\n##################')
    param = {}
    # param['booster'] = 'gbtree'
    #param['objective'] = 'binary:logistic'
    # param["eval_metric"] = "error"
    # param['eta'] = 0.3
    # param['gamma'] = 0
    param['max_depth'] = 6
    param['n_estimators'] = 200
    param['learning_rate'] = 0.1
    param['seed'] = 42
    # param['min_child_weight'] = 1
    # param['max_delta_step'] = 0
    # param['subsample'] = 1
    # param['colsample_bytree'] = 1
    # param['silent'] = 1
    # param['seed'] = 0
    # param['base_score'] = 0.4

    estimator = XGBClassifier()
    estimator.set_params(**param)
    x1, x2, y1, y2 = model_selection.train_test_split(df[features], df['Class'], test_size=0.2)
    calibratedCV = CalibratedClassifierCV(estimator, method='sigmoid', cv=5)
    calibratedCV.fit(x1, y1)
    prediction = calibratedCV.predict(x2)
    
#     print(calibratedCV.best_estimator_.booster())
#     print(calibratedCV.calibrated_classifiers_)
#     for c in calibratedCV.calibrated_classifiers_:
#         print(c.feature_importances_)
#         plot_importance(c)
#         pyplot.show()
#     feat_imp = pd.Series(calibratedCV.booster().get_fscore()).sort_values(ascending=False)
#     feat_imp.plot(kind='bar', title = 'Feature imp')
#     estimator.fit(x1, y1)
#     prediction = estimator.predict(x2)
#     xgb.plot_importance(estimator, max_num_features = 50)
#     imp = estimator.feature_importances_
#     plot_importance(estimator)
#     pyplot.show()

    acc = np.mean(prediction == y2)
    print('accuracy %s %s' % (acc, prediction[:5]))
    result = calibratedCV.predict(df_test[features])
#     result = estimator.predict(test)
#     return (imp, result)
    return result, acc





In [37]:
features = []

for nboverlap in range(20,301,20):
    for i in range(1, 10):
        columnName = 'PTextOverlap' + str(nboverlap) + 'C'+str(i)
        features.append(columnName)
prediction, _ = gboost1(train,test , features) # 0.633082706767 

submission(prediction)


##################
XGBoost
##################
accuracy 0.624060150376 [1 4 7 7 3]


NameError: name 'submission' is not defined

In [34]:
features = []

for nboverlap in range(20,301,20):
    for i in range(1, 10):
        columnName = 'PTextStemOverlap' + str(nboverlap) + 'C'+str(i)
        features.append(columnName)
prediction, _ = gboost1(train,test , features)


##################
XGBoost
##################
accuracy 0.615037593985 [7 7 7 1 7]


In [35]:
features = []
for nboverlap in range(20,301,20):
    for i in range(1, 10):
        columnName = 'PTextOverlapIntersect' + str(nboverlap) + 'C'+str(i)
        features.append(columnName)
prediction,_ = gboost1(train,test , features)


##################
XGBoost
##################
accuracy 0.621052631579 [7 4 4 4 1]


In [36]:
features = []
for nboverlap in range(20,301,20):
    for i in range(1, 10):
        columnName = 'PTextStemOverlapIntersect' + str(nboverlap) + 'C'+str(i)
        features.append(columnName)
prediction,_ = gboost1(train,test , features) #0.625563909774 


##################
XGBoost
##################


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\calibration.py:434: RuntimeWarning: divide by zero encountered in log
  l = -(T * np.log(P + tiny) + T1 * np.log(1. - P + tiny))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\calibration.py:434: RuntimeWarning: divide by zero encountered in log
  l = -(T * np.log(P + tiny) + T1 * np.log(1. - P + tiny))


accuracy 0.646616541353 [4 2 1 7 1]


In [ ]:
features = []
for nboverlap in range(20,301,20):
    for i in range(1, 10):
        columnName = 'PTextOverlapIntersect' + str(nboverlap) + 'C'+str(i)
        features.append(columnName)
        columnName = 'PTextOverlap' + str(nboverlap) + 'C'+str(i)
        features.append(columnName)
prediction,_ = gboost1(train,test ,features )


In [38]:
def submission(prediction):
    onehot = pd.get_dummies(prediction)
    submission = pd.DataFrame()
    submission['ID'] = test['ID']
    submission = submission.join(onehot)
    submission.columns = ['ID', 'class1','class2','class3','class4','class5','class6','class7','class8','class9']
    now = datetime.datetime.now()
    date_string = now.strftime('%Y-%m-%d-%I-%M')
    submission.to_csv('output/' + date_string + 'submission_xgb.csv', index=False)
submission(prediction)